## Práctica NASA

**Leemos el fichero**

In [1]:
fichero = open('C:\\Users\\nora.hafidi\\Desktop\\Big Data\\NASA\\NASA_access_log_Aug95\\access_log_Aug95')
nasa_ag = fichero.readlines()
fichero.close()

**Creamos la expresion regular**

In [2]:
expresion = '(.*)[\s-]{4}\s\[(\d\d)\/.*\/1995:(\d\d):(\d\d):(\d\d).*\s\"\s?([A-Z]*)\s(\/\S*).*\"\s?(\d\d\d)\s(\d*)'

In [ ]:
expresion_hive = '(.*)[\\s-]{4}\\s\\[(\\d\\d)\\/.*\\/1995:(\\d\\d):(\\d\\d):(\\d\\d).*\\s\\"\\s?([A-Z]*)\\s(\\/\\S*).*\\"\\s?(\\d\\d\\d)\\s(\\d*)'

In [3]:
import re
exp_fila=[]
for elem in nasa_ag:
    exp_fila.append(re.findall(expresion, elem))

In [4]:
print(len(exp_fila))

1569898


In [16]:
i=0
errores = 0
for elem in exp_fila:
    try:
        if len(elem[0]) != 9 or elem[0][8].isalpha():
            #print(i, elem)
            errores+=1
        else: pass
    except IndexError:
        errores+=1
        #print(i,elem)
    i+=1

In [18]:
errores

67

In [10]:
d = {}
for elem in exp_fila:
    try:
        if elem[0][7] in d:
            d[elem[0][7]] += 1
        else: d[elem[0][7]] = 1
    except IndexError: pass

In [11]:
d

{'200': 1398988,
 '304': 134146,
 '302': 26444,
 '404': 10052,
 '403': 171,
 '500': 3,
 '501': 27}

67 errores de un million está más que bien

**Creamos la tabla en Hive**

Movemos los datos desde la MV hasta HDFS con 
```hadoop fs -put /home/cloudera/Desktop/Datasets/nasa/NASA_access_log_Aug95/access_log_Aug95 /user/cloudera/nasa_ag/agosto95``` 
```hadoop fs -put /home/cloudera/Desktop/Datasets/nasa/NASA_access_log_Jul95/access_log_Jul95 /user/cloudera/nasa_jul/julio95``` y creamos una tabla externa con argumento location para no perder los datos. Luego la guardamos en formato parquet para poder utilizarla con impala.

agosto

```create external table aug95(host string, day string, hours string, minutes string, seconds string, request string, resource string, http_status_code int, size int) row format serde 'org.apache.hadoop.hive.serde2.RegexSerDe' with serdeproperties ("input.regex" = '(.*)[\\s-]{4}\\s\\[(\\d\\d)\\/.*\\/1995:(\\d\\d):(\\d\\d):(\\d\\d).*\\s\\"\\s?([A-Z]*)\\s(\\/\\S*).*\\"\\s?(\\d\\d\\d)\\s(\\d*)') location '/user/cloudera/nasa_ag/';```

```create table aug95_prima(host string, day string, hours string, minutes string, seconds string, request string, resource string, http_status_code int, size int) stored as parquet;```

```insert into table aug95_prima select * from aug95;```

julio

```create external table jul95(host string, day string, hours string, minutes string, seconds string, request string, resource string, http_status_code int, size int) row format serde 'org.apache.hadoop.hive.serde2.RegexSerDe' with serdeproperties ('input.regex' = '(.*)[\\s-]{4}\\s\\[(\\d\\d)\\/.*\\/1995:(\\d\\d):(\\d\\d):(\\d\\d).*\\s\\"\\s?([A-Z]*)\\s(\\/\\S*).*\\"\\s?(\\d\\d\\d)\\s(\\d*)', 'serialization.null.format' = '') location '/user/cloudera/nasa_jul/';```

```create table jul95_prima(host string, day string, hours string, minutes string, seconds string, request string, resource string, http_status_code int, size int) stored as parquet;```

```insert into table jul95_prima select * from jul95;```

Probamos un un ```select * from aug95 limit 5;``` y ```select * from aug95_prima limit 5;``` y vemos que parece que funcione bien

Veamos si los campos están buien o hay errores. Para ellos nos pasamos a impala que es bastante más rápido.

Hacemos un ```invalidate metadata``` en el impala-shell y ya podemos utilizar la base de datos y las tablas creadas en hive.

1. nulos: ```select count(*) from aug95 where variable is null;``` da 14.178 para todas las variables de la tabla
2. agrupar por request: ```select request from aug95_prima group by request;``` no hay errores aquí
3. numero de filas: ```select count(*) from aug95_prima;``` da 1.569.898 filas. **Veamos si coincide con las filas del documento.**

In [69]:
fichero_l = open('C:\\Users\\nora.hafidi\\Desktop\\Big Data\\NASA\\NASA_access_log_Aug95\\access_log_Aug95')
nasa_ag_lines = fichero_l.readlines()
fichero_l.close()

In [70]:
print(len(nasa_ag_lines))

1569898


Lo mismo con julio

1. nulos: Hay 16.830 nulos de cada columna. 16.000 de un millon no es significativo.
2. agrupar por request: ```select request from jul95_prima group by request;``` no hay errores aquí
3. numero de filas: ```select count(*) from jul95_prima;``` da 1.891.715 filas. **Veamos si coincide con las filas del documento.**

In [97]:
fichero_jul = open('C:\\Users\\nora.hafidi\\Desktop\\Big Data\\NASA\\NASA_access_log_Jul95\\access_log_Jul95', encoding='cp850')
nasa_jul = fichero_jul.readlines()
fichero_jul.close()

In [98]:
print(len(nasa_jul))

1891715


### Consultas

1. ¿Cuáles son los distintos protocolos web utilizados? Agrúpalos.
2. ¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos para ver cuál es el más común.
3. ¿Y los métodos de petición (verbos) más utilizados?
4. ¿Qué recurso tuvo la mayor transferencia de bytes de la página web?
5. Además, queremos saber que recurso de nuestra web es el que más tráfico recibe. Es decir, el recurso con más registros en nuestro log.
5. ¿Qué días la web recibió más tráfico?
6. ¿Cuáles son los hosts son los más frecuentes?
7. ¿A qué horas se produce el mayor número de tráfico en la web?
8. ¿Cuál es el número de errores 404 que ha habido cada día?

### Agosto

**¿Cuáles son los distintos protocolos web utilizados? Agrúpalos.**

Aquí he utilizado una expresión regular diferente que me leía los protocolos, pero como en las instrucciones indica que todos los protocolos son HTTP/1.0 he decidido no leerlo para añadir 3.000 filas en la tabla.

```select protocol from aug95_prima group by protocol;```


| protocol | count(*) |
----------|----------
| NULL     | 16830    |
| HTTP/1.0 | 1553068  |



**¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos para ver cuál es el más común.**

```select http_status_code, count(*) as total from aug95_prima group by http_status_code order by total desc limit 1;```

| http_status_code | total   |
|------------------|---------|
| 200              | 1398910 |


**¿Y los métodos de petición (verbos) más utilizados?**

```select request, count(*) as total from aug95_prima group by request order by total desc limit 1;```

| request | total   |
|---------|---------|
| GET     | 1551681 |


**¿Qué recurso tuvo la mayor transferencia de bytes de la página web?**

```select * from aug95_prima where resource is not null order by resource desc limit 1;```

| host                 | day | hours | minutes | seconds | request | resource             | http_status_code | size  |
|----------------------|-----|-------|---------|---------|---------|----------------------|------------------|-------|
| tide01.microsoft.com | 03  | 15    | 44      | 30      | GET     | /~downs/launchup.gif | 200              | 53409 |



**Además, queremos saber qué recurso de nuestra web es el que más tráfico recibe. Es decir, el recurso con más registros en nuestro log.**

```select * from aug95_prima where size is not null order by size desc limit 1;```

| host           | day | hours | minutes | seconds | request | resource  | http_status_code | size    |
|----------------|-----|-------|---------|---------|---------|----|------------------|---------|
| 163.205.156.16 | 03  | 15    | 51      | 23      | GET     | /statistics/1995/Jul/Jul95_reverse_domains.html | 200              | 3421948 |



**¿Qué días la web recibió más tráfico?**

```select day, count(*) as total from aug95_prima where day is not null group by day order by total desc limit 3;```

| day | total |
|-----|-------|
| 31  | 88942 |
| 30  | 79404 |
| 29  | 67244 |


**¿Cuáles son los hosts más frecuentes?**

```select host, count(*) as total from aug95_prima where host is not null group by host order by total desc limit 5;```

| host                 | total |
|----------------------|-------|
| edams.ksc.nasa.gov   | 6515  |
| piweba4y.prodigy.com | 4797  |
| 163.206.89.4         | 4769  |
| piweba5y.prodigy.com | 4564  |
| piweba3y.prodigy.com | 4354  |

**¿A qué horas se produce el mayor número de tráfico en la web?**

```select hours, sum(size) as traf from aug95_prima where hours is not null group by hours order by traf desc limit 5```

| hours | traf       |
|-------|------------|
| 13    | 1797569666 |
| 15    | 1762625783 |
| 12    | 1757290922 |
| 16    | 1725490692 |
| 14    | 1636102102 |


**¿Cuál es el número de errores 404 que ha habido cada día?**

```select day, count(*) from aug95_prima where http_status_code = '404' group by day```

### Julio

**¿Cuáles son los distintos protocolos web utilizados? Agrúpalos.**

**¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos para ver cuál es el más común.**

```select http_status_code, count(*) as total from jul95_prima group by http_status_code order by total desc limit 1;```

| http_status_code | total   |
|------------------|---------|
| 200              | 1701451 |


**¿Y los métodos de petición (verbos) más utilizados?**

```select request, count(*) as total from aug95_prima group by request order by total desc limit 1;```

**¿Qué recurso tuvo la mayor transferencia de bytes de la página web?**

**Además, queremos saber qué recurso de nuestra web es el que más tráfico recibe. Es decir, el recurso con más registros en nuestro log.**

**¿Qué días la web recibió más tráfico?**

**¿Cuáles son los hosts más frecuentes?**

**¿A qué horas se produce el mayor número de tráfico en la web?**

**¿Cuál es el número de errores 404 que ha habido cada día?**